In [69]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
import lightgbm
from sklearn.metrics import mean_squared_error
import warnings
import gc


warnings.filterwarnings('ignore')
np.random.seed(4590)

In [70]:
sales_train = pd.read_csv('/Users/sinsakuokazaki/Project/pred-future-sales/\
input/competitive-data-science-predict-future-sales/sales_train.csv')

In [71]:
test = pd.read_csv('/Users/sinsakuokazaki/Project/pred-future-sales/\
input/competitive-data-science-predict-future-sales/test.csv')

In [72]:
train = pd.read_csv('/Users/sinsakuokazaki/Project/pred-future-sales/\
input/competitive-data-science-predict-future-sales/train.csv')

In [73]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,dayofweek,dayofmonth,month,quarter,year,minute,hour,total_sales,shop_name,item_name,item_category_id,item_category_name
0,2013-01-02,0,59,22154,999.00,1.0,2,2,1,1,2013,0,0,999.00,"Ярославль ТЦ ""Альтаир""",ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray
1,2013-01-03,0,25,2552,899.00,1.0,3,3,1,1,2013,0,0,899.00,"Москва ТРК ""Атриум""",DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил
2,2013-01-05,0,25,2552,899.00,-1.0,5,5,1,1,2013,0,0,-899.00,"Москва ТРК ""Атриум""",DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил
3,2013-01-06,0,25,2554,1709.05,1.0,6,6,1,1,2013,0,0,1709.05,"Москва ТРК ""Атриум""",DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил
4,2013-01-15,0,25,2555,1099.00,1.0,1,15,1,1,2013,0,0,1099.00,"Москва ТРК ""Атриум""",DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства


In [100]:
data = sales_train.drop(columns = ["date", 'minute', 'hour', 'total_sales','shop_name', 
                         'item_name', 'item_category_name', 'dayofweek', 'dayofmonth', 'item_cnt_day'])

In [101]:
data.head()

,date_block_num,shop_id,item_id,item_price,month,quarter,year,item_category_id
0,0,59,22154,999.00,1,1,2013,37
1,0,25,2552,899.00,1,1,2013,58
2,0,25,2552,899.00,1,1,2013,58
3,0,25,2554,1709.05,1,1,2013,58
4,0,25,2555,1099.00,1,1,2013,56


In [102]:
test["date_block_num"] = 34

In [103]:
data = pd.concat([data, test.drop(columns=['item_name', 'item_category_name',
                                           'shop_name', 'in_train', 'combination_id'])], join='outer')

In [104]:
data = data.fillna(0)

In [105]:
data.head()

,ID,date_block_num,item_category_id,item_id,item_price,month,quarter,shop_id,year
0,0.0,0,37,22154,999.00,1,1,59,2013
1,0.0,0,58,2552,899.00,1,1,25,2013
2,0.0,0,58,2552,899.00,1,1,25,2013
3,0.0,0,58,2554,1709.05,1,1,25,2013
4,0.0,0,56,2555,1099.00,1,1,25,2013


In [106]:
data.tail()

,ID,date_block_num,item_category_id,item_id,item_price,month,quarter,shop_id,year
214195,214195.0,34,55,18454,0.0,11,4,45,2015
214196,214196.0,34,64,16188,0.0,11,4,45,2015
214197,214197.0,34,55,15757,0.0,11,4,45,2015
214198,214198.0,34,40,19648,0.0,11,4,45,2015
214199,214199.0,34,37,969,0.0,11,4,45,2015


In [107]:
def mean_encode(statistic, group, col_name, data):
    group_df = data.groupby(group).agg({'item_price': [statistic]})
    column = col_name.format(statistic)
    group_df.columns = [ column ]
    group_df.reset_index(inplace=True)
    return group_df, column

def lag_mean_encoding(lag, group_df, column, group, data):
    tmp = group_df.copy()
    tmp['date_block_num'] += lag
    new_name = column + str(lag)
    tmp = tmp.rename(index=str, columns={column: new_name})
    print(new_name)
    data = pd.merge(data, tmp, on=group, how='left')
    return data

In [108]:
groups = [["date_block_num"], ["date_block_num", "item_id"], ["date_block_num", "shop_id"],
         ["date_block_num", "item_category_id"], ["date_block_num", "item_category_id", "shop_id"]]

col_names = ['date_{}_item_cnt', 'date_item_{}_item_cnt', 'date_shop_{}_item_cnt',
             'date_category_{}_item_cnt', 'date_category_shop_{}_item_cnt']

for s in ["mean", "std", "median"]:
    for g, c in zip(groups, col_names):
        group_df, column = mean_encode(s, g, c, data)
        for l in [2, 3, 6, 12]:
            data = lag_mean_encoding(l, group_df, column, g, data)

date_mean_item_cnt2
date_mean_item_cnt3
date_mean_item_cnt6
date_mean_item_cnt12
date_item_mean_item_cnt2
date_item_mean_item_cnt3
date_item_mean_item_cnt6
date_item_mean_item_cnt12
date_shop_mean_item_cnt2
date_shop_mean_item_cnt3
date_shop_mean_item_cnt6
date_shop_mean_item_cnt12
date_category_mean_item_cnt2
date_category_mean_item_cnt3
date_category_mean_item_cnt6
date_category_mean_item_cnt12
date_category_shop_mean_item_cnt2
date_category_shop_mean_item_cnt3
date_category_shop_mean_item_cnt6
date_category_shop_mean_item_cnt12
date_std_item_cnt2
date_std_item_cnt3
date_std_item_cnt6
date_std_item_cnt12
date_item_std_item_cnt2
date_item_std_item_cnt3
date_item_std_item_cnt6
date_item_std_item_cnt12
date_shop_std_item_cnt2
date_shop_std_item_cnt3
date_shop_std_item_cnt6
date_shop_std_item_cnt12
date_category_std_item_cnt2
date_category_std_item_cnt3
date_category_std_item_cnt6
date_category_std_item_cnt12
date_category_shop_std_item_cnt2
date_category_shop_std_item_cnt3
date_category

In [109]:
data = data.fillna(0)
data = data[data['date_block_num'] > 11]

In [112]:
data = data.drop(columns=['ID'])

In [113]:
data.columns

Index(['date_block_num', 'item_category_id', 'item_id', 'item_price', 'month',
       'quarter', 'shop_id', 'year', 'date_mean_item_cnt2',
       'date_mean_item_cnt3', 'date_mean_item_cnt6', 'date_mean_item_cnt12',
       'date_item_mean_item_cnt2', 'date_item_mean_item_cnt3',
       'date_item_mean_item_cnt6', 'date_item_mean_item_cnt12',
       'date_shop_mean_item_cnt2', 'date_shop_mean_item_cnt3',
       'date_shop_mean_item_cnt6', 'date_shop_mean_item_cnt12',
       'date_category_mean_item_cnt2', 'date_category_mean_item_cnt3',
       'date_category_mean_item_cnt6', 'date_category_mean_item_cnt12',
       'date_category_shop_mean_item_cnt2',
       'date_category_shop_mean_item_cnt3',
       'date_category_shop_mean_item_cnt6',
       'date_category_shop_mean_item_cnt12', 'date_std_item_cnt2',
       'date_std_item_cnt3', 'date_std_item_cnt6', 'date_std_item_cnt12',
       'date_item_std_item_cnt2', 'date_item_std_item_cnt3',
       'date_item_std_item_cnt6', 'date_item_std_item

In [118]:
data = data[data['item_price'] < 100000]

In [119]:
X_tra = data[data['date_block_num'] < 33].drop(columns = ['item_price'])
Y_tra = data[data['date_block_num'] < 33]['item_price']

X_val = data[data['date_block_num'] == 33].drop(columns = ['item_price'])
Y_val = data[data['date_block_num'] == 33]['item_price']

X_test = data[data['date_block_num'] == 34]['item_price']

In [120]:
params = {"max_depth": 7,
         "min_child_weight": 15,
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.01,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": 4,
         "random_state": 4590}

In [121]:
tra_data = lightgbm.Dataset(X_tra, label=Y_tra)
val_data = lightgbm.Dataset(X_val, label=Y_val)

model = lightgbm.train(params, tra_data, num_boost_round=1000, \
                           valid_sets=[tra_data, val_data],  verbose_eval=100, \
                           early_stopping_rounds = 100)
pred = model.predict(X_val, num_iteration=model.best_iteration)
np.sqrt(mean_squared_error(pred, Y_val))

Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 841.01	valid_1's rmse: 1163.97
[200]	training's rmse: 495.651	valid_1's rmse: 843.057
[300]	training's rmse: 407.27	valid_1's rmse: 772.327
[400]	training's rmse: 379.295	valid_1's rmse: 753.53
[500]	training's rmse: 366.002	valid_1's rmse: 747.883
[600]	training's rmse: 356.045	valid_1's rmse: 749.924
Early stopping, best iteration is:
[521]	training's rmse: 363.77	valid_1's rmse: 747.398


747.3980548883537